# Explore biosamples

In [1]:
import sys
import asyncio
import aiohttp
import logging
from pprint import pprint

sys.path.insert(0, "../scripts")
logger = logging.getLogger(__name__)

# custom supercronic modules
from helpers.backend import CONNECTOR, get_ruleset, CDPConverter
from helpers.biosamples import get_biosample_record

In [2]:
async def get_converter(session):
    # Get rules from github. Start a task
    logger.info("Getting ruleset")
    ruleset_task = asyncio.create_task(get_ruleset(session))
    standard_rules, organism_rules, specimen_rules = await ruleset_task
    converter = CDPConverter(
            standard_rules,
            organism_rules,
            specimen_rules)
        
    return converter

In [3]:
session = aiohttp.ClientSession(connector=CONNECTOR)
loop = asyncio.get_event_loop()
converter = await loop.create_task(get_converter(session))
biosample_record, etag = await loop.create_task(get_biosample_record('SAMEA7044752', session))
session.close()

<coroutine object ClientSession.close at 0x7f3176787200>

In [4]:
converter.convert_record(biosample_record, etag)

Unmanaged key 'externalReferences': [{'url': 'https://www.ebi.ac.uk/ena/data/view/ERS5331193', 'duo': []}]


{'data_source_id': 'SAMEA7044752',
 'alternative_id': '1-FLI_IM_661522',
 'project': 'IMAGE',
 'submission_title': 'Other chicken',
 'material': 'organism',
 'material_ontology': 'http://purl.obolibrary.org/obo/OBI_0100026',
 'person_last_name': ['Garcia'],
 'person_email': ['mailto:robin.garcia@fli.de'],
 'person_affiliation': ['Friedrich Loeffler Institut - Bundesforschungsinstitut fuer Tiergesundheit'],
 'person_role': ['submitter'],
 'person_role_ontology': ['http://www.ebi.ac.uk/efo/EFO_0001741'],
 'organization_name': ['Friedrich Loeffler Institut - Bundesforschungsinstitut fuer Tiergesundheit'],
 'organization_role': ['submitter'],
 'organization_role_ontology': ['http://www.ebi.ac.uk/efo/EFO_0001741'],
 'gene_bank_name': 'FLI',
 'gene_bank_country': 'Germany',
 'gene_bank_country_ontology': 'http://purl.obolibrary.org/obo/NCIT_C16636',
 'data_source_type': 'Template',
 'data_source_version': '2020-05-19, version 1.5',
 'species': 'Gallus gallus',
 'species_ontology': 'http://pu

In [5]:
biosample_record.keys()

dict_keys(['name', 'accession', 'domain', 'release', 'update', 'taxId', 'characteristics', 'relationships', 'externalReferences', 'releaseDate', 'updateDate', 'submittedVia', 'create', 'certificates', '_links'])